In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, metrics, cv

In [2]:
train_df = pd.read_csv('data/train_with_pred.csv')
test_df = pd.read_csv('data/test_with_pred.csv')

In [3]:
temp_x = train_df.drop(labels=['Id','SalePrice'],axis=1)
Y = train_df.SalePrice
categorical_values = np.append([1],np.where(temp_x.dtypes == 'object')[0])

In [4]:
X = temp_x
X["NumPred"] = temp_x["NumPred"]
X

C:\Users\gonza\AppData\Local\Temp\ipykernel_16336\797750040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["NumPred"] = temp_x["NumPred"]


,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,NumPred
0,60,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,RFn,TA,TA,Y,NO,NO,NO,WD,Normal,225478.39
1,20,RL,Pave,NO,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,RFn,TA,TA,Y,NO,NO,NO,WD,Normal,187820.31
2,60,RL,Pave,NO,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,RFn,TA,TA,Y,NO,NO,NO,WD,Normal,213078.67
3,70,RL,Pave,NO,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,Unf,TA,TA,Y,NO,NO,NO,WD,Abnorml,149093.75
4,60,RL,Pave,NO,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,RFn,TA,TA,Y,NO,NO,NO,WD,Normal,290045.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,RFn,TA,TA,Y,NO,NO,NO,WD,Normal,177545.75
1456,20,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,Unf,TA,TA,Y,NO,MnPrv,NO,WD,Normal,229720.55
1457,70,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,RFn,TA,TA,Y,NO,GdPrv,Shed,WD,Normal,229899.02
1458,20,RL,Pave,NO,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,Unf,TA,TA,Y,NO,NO,NO,WD,Normal,135878.25


In [9]:

# Setup The catboost model
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, train_size=0.8, random_state=33)
model = CatBoostRegressor(
    loss_function='RMSE',
    num_trees=470,
    depth=11,
    random_seed=33,
    use_best_model=True
)

In [10]:
# Train the model
model.fit(
    X_train, y_train,
    cat_features=np.append([1],np.where(X.dtypes == 'object')[0]),
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.0827
0:	learn: 75036.1736832	test: 73984.3240706	best: 73984.3240706 (0)	total: 53.9ms	remaining: 25.3s
1:	learn: 70994.8551158	test: 69468.0445511	best: 69468.0445511 (1)	total: 98.5ms	remaining: 23s
2:	learn: 66962.5418076	test: 65114.5243021	best: 65114.5243021 (2)	total: 114ms	remaining: 17.8s
3:	learn: 63672.5059914	test: 62076.1954976	best: 62076.1954976 (3)	total: 179ms	remaining: 20.8s
4:	learn: 60389.0842799	test: 58523.2423184	best: 58523.2423184 (4)	total: 218ms	remaining: 20.3s
5:	learn: 57188.5727458	test: 55067.2335477	best: 55067.2335477 (5)	total: 240ms	remaining: 18.6s
6:	learn: 54165.8172498	test: 51954.6509672	best: 51954.6509672 (6)	total: 259ms	remaining: 17.1s
7:	learn: 51669.9245209	test: 49281.7582932	best: 49281.7582932 (7)	total: 280ms	remaining: 16.2s
8:	learn: 49418.2598053	test: 47229.0351298	best: 47229.0351298 (8)	total: 412ms	remaining: 21.1s
9:	learn: 47534.1827261	test: 45080.4242516	best: 45080.4242516 (9)	total: 423ms	remaining

In [17]:
test_df.drop('Id',axis=1)
temp_test_x = test_df.drop(labels=['Id'],axis=1)
categorical_values = np.append([1],np.where(temp_test_x.dtypes == 'object')[0])
test_X = temp_test_x.iloc[:,categorical_values]
test_X["NumPred"] = temp_test_x["NumPred"]
test_df

C:\Users\gonza\AppData\Local\Temp\ipykernel_16336\3366227594.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X["NumPred"] = temp_test_x["NumPred"]


,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,NumPred
0,0,1461,20,RH,80.0,11622,Pave,NO,Reg,Lvl,...,0,NO,MnPrv,NO,0,6,2010,WD,Normal,119405.516
1,1,1462,20,RL,81.0,14267,Pave,NO,IR1,Lvl,...,0,NO,NO,Gar2,12500,6,2010,WD,Normal,163346.270
2,2,1463,60,RL,74.0,13830,Pave,NO,IR1,Lvl,...,0,NO,MnPrv,NO,0,3,2010,WD,Normal,190840.780
3,3,1464,60,RL,78.0,9978,Pave,NO,IR1,Lvl,...,0,NO,NO,NO,0,6,2010,WD,Normal,204806.080
4,4,1465,120,RL,43.0,5005,Pave,NO,IR1,HLS,...,0,NO,NO,NO,0,1,2010,WD,Normal,176631.890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1454,2915,160,RM,21.0,1936,Pave,NO,Reg,Lvl,...,0,NO,NO,NO,0,6,2006,WD,Normal,106364.320
1455,1455,2916,160,RM,21.0,1894,Pave,NO,Reg,Lvl,...,0,NO,NO,NO,0,4,2006,WD,Abnorml,93830.710
1456,1456,2917,20,RL,160.0,20000,Pave,NO,Reg,Lvl,...,0,NO,NO,NO,0,9,2006,WD,Abnorml,198141.170
1457,1457,2918,85,RL,62.0,10441,Pave,NO,Reg,Lvl,...,0,NO,MnPrv,Shed,700,7,2006,WD,Normal,99047.414


In [18]:
predictions = model.predict(test_X)
final_df = pd.DataFrame({'Id':test_df['Id'],'SalePrice':predictions})
final_df.to_csv('data/submision2.csv',index=False)